In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Загружаем функцию для генерирования полиномиальных признаков:
from sklearn.preprocessing import PolynomialFeatures

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# Any results you write to the current directory are saved as output.
import re #регулярные выражения
from datetime import datetime as dat #работа с датами

from random import randint





# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [77]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 36

In [78]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [79]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [80]:
df_train.info()

In [81]:
df_train.sample(5)

In [82]:
df_test.info()

In [83]:
df_test.sample(5)

In [84]:
sample_submission.head(5)

In [85]:
sample_submission.info()

In [86]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем, где у нас обучение
df_test['sample'] = 0 # помечаем, где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, поэтому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [87]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [88]:
data.sample(5)

In [89]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
<!-- ![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png) -->

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

Приведем названия столбцов датасета к более лаконичному виду, удалим ненужные столбцы:

In [90]:
data.sample()

In [91]:
data.drop(['URL_TA', 'ID_TA'], axis=1, inplace=True)
data.columns = ['id', 'city', 'cuisine', 'ranking', 'price_range', 'rev_number', 'reviews', 'sample', 'rating']

In [92]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['rev_number']).astype('uint8')

In [93]:
data['Number_of_Reviews_isNAN']

In [94]:
# Заполняем пропуски средним по городу

mean_NoR_by_city = dict([
      (city, data.loc[data.loc[:, 'city'] == city, 'rev_number'].mean())\
      for city in data.loc[:, 'city'].unique()
])

data.loc[:, 'rev_number'].\
    fillna(data.loc[:, 'city'].\
           apply(lambda c: mean_NoR_by_city[c]), inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [95]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
<!-- ![](https://i.imgur.com/mtimFxh.png) -->

In [96]:
#One-Hot Encoding для топа городов
top_cities_num = 5 #можно установить любое
top_cities = data.loc[:, 'city'].value_counts()[0:top_cities_num].index
top_cities = list(map(lambda x: '_' + x, top_cities))
data = pd.concat([
         data,
         pd.get_dummies(data, columns=['city'], prefix='').
         loc[:, top_cities]],
         axis=1
)

In [97]:
data.head(5)

In [98]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [99]:
data['price_range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [100]:
# моя обработка 'Price Range'
price_range_quantize = {
    '$': 1, '$$ - $$$': 2, '$$$$': 3
}
data.loc[:, 'price_range']=\
data.loc[:, 'price_range'].apply(lambda x: 0 if pd.isna(x) else price_range_quantize[x])

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [101]:
# тут ваш код на обработку других признаков
# обработка Cuisine Style
# превращаем значения Cuisine Style из строк в списки со строковыми элементами
data.loc[:, 'cuisine'] =\
data.loc[:, 'cuisine'].fillna(value='[]').apply(
    lambda cuisine_list_for_id: \
    [s.replace("'", "") for s in cuisine_list_for_id[2:-2].split(', ')]
)

In [102]:
# обработка Reviews
# функция для извлечения дат двух последних отзывов
def rev_dates(s):
    date_regex = re.compile(r'(\d+/\d+/\d+)') # компилируем регулярное выражение для поиска дат в формате dd/mm/YYYY
    str_date_ls = date_regex.findall(s)
    if str_date_ls != []:
        enum = list(enumerate(str_date_ls))
        f = ['%m/%d/%Y' if int(e[1][0:2].replace('/', '')) <= 12 else '%d/%m/%Y' for e in enum]
        return [dat.strptime(e[1], f[e[0]]) for e in enum]
    else:
        return [dat(1970, 1, 1, 0, 0)] # в случае если не нашли дату

# принимает список дат;
# возвращает разность в днях между двумя последними отзывами
# если дата только одна, возвращается 0
def last_rev_date_delta(dates):
    if len(dates) >= 3:
        return abs(dates[2] - dates[1]).days
    else:
        return abs(dates[0] - dates[-1]).days
    


# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
<!-- ![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png) -->

### Посмотрим распределение признака Ranking

In [103]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100);

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [104]:
df_train['City'].value_counts(ascending=True).plot(kind='barh');

In [105]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100);

In [106]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=80)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.


### Посмотрим распределение целевой переменной

In [107]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh');

### ***Целевая переменная принимает значения, только кратные 0.5, что можно будет использовать для улучшения метрики предсказания***

### Посмотрим распределение целевой переменной относительно признака

In [108]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100);

In [109]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100);

In [110]:
df_train['Ranking'].value_counts()

### Тепловая карта корреляции признаков

In [111]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot = True, cmap = 'Spectral_r');

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [112]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [113]:
data.Reviews.apply(type).value_counts()
data.Reviews[data.Reviews.apply(type) == float]

In [114]:
# класс для предобработки данных датасета для модели
class PreprocData:
    '''includes several functions to pre-process the predictor data.'''
    
    def __init__(self, df_input):
        self.data = df_input.copy()

    def preprocess(self):
        # ################### 1. Предобработка ############################################################## 
        # убираем не нужные для модели признаки
        self.data.drop(['URL_TA', 'ID_TA'], axis=1, inplace=True)
        self.data.columns = ['id', 'city', 'cuisine', 'ranking', 'price_range', 'rev_number', 'reviews', 'sample', 'rating']

        # обработка Cuisine Style
        # превращаем значения Cuisine Style из строк в списки со строковыми элементами
        self.data.loc[:, 'cuisine'] =\
        self.data.loc[:, 'cuisine'].fillna(value='[]').apply(
            lambda cuisine_list_for_id: \
            [s.replace("'", "") for s in cuisine_list_for_id[2:-2].split(', ')]
        )
    
        # обработка Reviews
        # функция для извлечения дат двух последних отзывов
        # принимает строку, возвращает список объектов datetime
        # если в строке не удалось найти дату, возвращает список с единственным
        # datetime 01.01.1970 00:00
        def rev_dates(s):
            date_regex = re.compile(r'(\d+/\d+/\d+)') # компилируем регулярное выражение для поиска дат в формате dd/mm/YYYY
            str_date_ls = date_regex.findall(s)
            if str_date_ls != []:
                enum = list(enumerate(str_date_ls))
                f = ['%m/%d/%Y' if int(e[1][0:2].replace('/', '')) <= 12 \
                     else '%d/%m/%Y' if '/' not in e[1][-4:-1] else '%d/%m/%y' for e in enum]
                return [dat.strptime(e[1], f[e[0]]) for e in enum]
            else:
                return [dat(1970, 1, 1, 0, 0)] # в случае если не нашли дату
            
        # добавляем в данные столбец с кол-вом дней между двумя последними отзывами
        self.data.loc[:, 'reviews'].fillna('', inplace=True)
        self.data.loc[:, 'rev_dates'] =\
            self.data.loc[:, 'reviews'].apply(lambda s: rev_dates(s))
        
    
    def fill_missing(self):
        
    # ################### 2. NAN ############################################################## 
        # Заполняем пропуски в Number of Reviews средним по городу

        mean_NoR_by_city = dict([
          (city, self.data.loc[self.data.loc[:, 'city'] == city, 'rev_number'].mean())\
          for city in self.data.loc[:, 'city'].unique()
        ])

        self.data.loc[:, 'rev_number'].\
            fillna(self.data.loc[:, 'city'].\
               apply(lambda c: mean_NoR_by_city[c]), inplace=True)
    #==============================================================================================
    #заполняем пропуски в price_range - сохраняя ценовое распределение ресторанов в каждом городе

    #     def fill_price(city, city_pr_distr): # distr -> distr[city][distr[city].index != 0]
    #         borders = [city_pr_distr[0:i+1].sum() for i in range(len(city_pr_distr))]
        
    #         pick_rand = randint(0, borders[-1])
    #         for border in enumerate(borders):
    #             if pick_rand < border[1]:
    #                 return city_pr_distr.index[border[0]]

    #     distr = self.data.groupby('city')['price_range'].value_counts()

    #     self.data.loc[dself.data.loc[:, 'price_range'] == 0, 'price_range'] = \
    #     self.data.loc[self.data.loc[:, 'price_range'] == 0, 'city'].apply(
    #         lambda city: fill_price(city, distr[city][distr[city].index != 0])
    #     )


    #     self.data.loc[self.data.loc[:, 'price_range'].apply(pd.isna), 'price_range'] = \
    #     self.data.loc[self.data.loc[:, 'price_range'].apply(pd.isna), 'city'].apply(
    #         lambda city: fill_price(city, distr[city][distr[city].index != 0])
    #     )
    
    #     подобное заполнение Price Range себя не оправдало - заполнение нулями дает лучший результат

    def encode(self):
    # ################### 3. Encoding ############################################################## 
        #One-Hot Encoding для топа городов
        
        top_cities_num = 5 #можно установить любое
        top_cities = self.data.loc[:, 'city'].value_counts()[0:top_cities_num].index
        top_cities = list(map(lambda x: '_' + x, top_cities))
        self.data = pd.concat([
             self.data,
             pd.get_dummies(self.data, columns=['city'], prefix='').
             loc[:, top_cities]],
             axis=1
        )
    #==============================================================================================
        # One-Hot Encoding для Сuisine Style и прикрепляем его к осн. датафрейму
        
        top_cuisines_num = 5  #кол-во самых распространненных кухонь, можно взять любое

        # создаем список самых популярных top_cuisines_num типов кухонь
        top_cuisines = self.data.loc[self.data.loc[:, 'cuisine'].apply(lambda x: x != ['']), :].\
            explode('cuisine').groupby('cuisine')['cuisine'].\
                count().sort_values(ascending=False).head(top_cuisines_num).index

        # прикрепляем к датафрейму
        top_cuisines = list(map(lambda x: '_' + x, top_cuisines))
        self.data = pd.concat([
             self.data,
             pd.get_dummies(self.data.explode('cuisine'), columns=['cuisine'], prefix='').
             sum(level=0).loc[:, top_cuisines]],
             axis=1
        )
        #==============================================================================================    
        # Label Encoding для Price Range
    #     price_range_encode = {
    #         '$': 1, '$$ - $$$': 2, '$$$$': 3
    #     }
    #     self.data.loc[:, 'price_range']=\
    #         self.data.loc[:, 'price_range'].apply(lambda x: 0 if pd.isna(x) else price_range_encode)
    #==============================================================================================    
        # One-Hot Encoding для Price Range

        price_range_rename = {
            '$': 'S', '$$ - $$$': 'SSS', '$$$$': 'SSSS' 
        }
        self.data.loc[:, 'price_range'].fillna('NaN', inplace=True)
        self.data.loc[:, 'price_range']=\
        self.data.loc[:, 'price_range'].apply(lambda x: price_range_rename[x] if x !='NaN' else 'NaN')
    
        self.data = pd.get_dummies(self.data, columns=['price_range'], prefix='pr', dummy_na=True) 
    
    def add_features(self):
    # ################### 4. Feature Engineering ####################################################
    # код на генерацию новых признаков
    # добавляем признак - численность населения города, в котором находится ресторан
    # данные - из Википедии (население не включающее городскую агломерацию)
        
        # добавляем в данные признак - кол-во дней между двумя последними отзывами
        
        # принимает список дат;
        # возвращает разность в днях между двумя последними отзывами
        # если дата только одна, возвращается 0
        def last_rev_date_delta(dates):
            if len(dates) >= 3:
                return abs(dates[2] - dates[1]).days
            else:
                return abs(dates[0] - dates[-1]).days
            
        self.data.loc[:, 'delta'] =\
            self.data.loc[:, 'rev_dates'].apply(lambda s: last_rev_date_delta(s))
    
        population = {
           'Paris': 2176, 'Stockholm': 1656, 'London': 10840, 'Berlin': 3664,
           'Munich': 1553, 'Oporto': 232, 'Milan': 3144, 'Bratislava': 433,
           'Vienna': 1911, 'Rome': 2860, 'Barcelona': 1620, 'Madrid': 3223,
           'Dublin': 555, 'Brussels': 1209, 'Zurich': 415, 'Warsaw': 1794,
           'Budapest': 1752, 'Copenhagen': 799, 'Amsterdam': 873, 'Lyon': 516,
           'Hamburg': 1899, 'Lisbon': 507, 'Prague': 1324, 'Oslo': 697, 
           'Helsinki': 658, 'Edinburgh': 488, 'Geneva': 202, 'Ljubljana': 296,
           'Athens': 664, 'Luxembourg': 125, 'Krakow': 781}
        self.data.loc[:, 'population'] = self.data.loc[:, 'city'].apply(lambda city: population[city])
    #==============================================================================================  
        # добавляем признак - является ли город столицей: 1 - да, 0 - нет
    
        is_capital = {'Paris': 1, 'Stockholm': 1, 'London': 1, 'Berlin': 1,
           'Munich': 0, 'Oporto': 0, 'Milan': 0, 'Bratislava': 1,
           'Vienna': 1, 'Rome': 1, 'Barcelona': 0, 'Madrid': 1,
           'Dublin': 1, 'Brussels': 1, 'Zurich': 0, 'Warsaw': 1,
           'Budapest': 1, 'Copenhagen': 1, 'Amsterdam': 1, 'Lyon': 0,
           'Hamburg': 0, 'Lisbon': 1, 'Prague': 1, 'Oslo': 1, 
           'Helsinki': 1, 'Edinburgh': 0, 'Geneva': 0, 'Ljubljana': 1,
           'Athens': 1, 'Luxembourg': 1, 'Krakow': 0
        }
        self.data.loc[:, 'is_capital'] = self.data.loc[:, 'city'].apply(lambda city: is_capital[city])
    #==============================================================================================    
        # добавляем признак - с подушевым ВВП по ППС города, в кот. нах. ресторан
        # данные взяты из https://ec.europa.eu/eurostat/
    
        gdp = {
            'Paris': 61883, 'Stockholm': 61754, 'London': 58827, 'Berlin': 37601,
            'Munich': 69844, 'Oporto': 24819, 'Milan': 51768, 'Bratislava': 67841,
            'Vienna': 46787, 'Rome': 41475, 'Barcelona': 45752, 'Madrid': 43074,
            'Dublin':30000, 'Brussels': 54634, 'Zurich': 64302, 'Warsaw': 49722,
            'Budapest': 37399, 'Copenhagen': 54197, 'Amsterdam': 60857, 'Lyon': 46913,
            'Hamburg': 52947, 'Lisbon': 34782, 'Prague': 48160, 'Oslo': 64673,
            'Helsinki': 49760, 'Edinburgh': 44059, 'Geneva': 62115, 'Ljubljana': 39763,
            'Athens': 32167, 'Luxembourg': 88312, 'Krakow': 29695
        }
        self.data.loc[:, 'gdp'] = self.data.loc[:, 'city'].apply(lambda city: gdp[city])
    #==============================================================================================
        #добавляем признак - количество предприятий общепита в стране нахождения ресторана
        # данные взяты из https://ec.europa.eu/eurostat/ 
        n_of_food_serv = {
            'Paris': 161466, 'Stockholm': 23208, 'London': 88841, 'Berlin': 136091,
            'Munich': 136091, 'Oporto': 31363, 'Milan': 155875, 'Bratislava': 10917,
            'Vienna': 27348, 'Rome': 155875, 'Barcelona': 72657, 'Madrid': 72657,
            'Dublin': 7993, 'Brussels': 30773, 'Zurich': 11961, 'Warsaw': 39291,
            'Budapest': 16850, 'Copenhagen': 8983, 'Amsterdam': 30502, 'Lyon': 161466,
            'Hamburg': 136091, 'Lisbon': 31363, 'Prague': 45623, 'Oslo': 6605,
            'Helsinki': 7769, 'Edinburgh': 88841, 'Geneva': 11961, 'Ljubljana': 4369,
            'Athens': 39978, 'Luxembourg': 1475, 'Krakow': 39291
        }
        self.data.loc[:, 'n_of_food_serv'] = self.data.loc[:, 'city'].\
            apply(lambda city: n_of_food_serv[city])
    #==============================================================================================
        #количество ресторанов в городе согласно основному датасету на душу населения города
        n_of_rest = self.data.loc[:, 'city'].value_counts()

        self.data.loc[:, 'n_of_rest'] = self.data.loc[:, 'city'].\
            apply(lambda city: n_of_rest[city])

        self.data.loc[:, 'n_of_rest_per_capita'] = self.data.loc[:, 'n_of_rest'] /\
            self.data.loc[:, 'population']
    #==============================================================================================
        # добавляем признак суммарного ranking по городам
    
        ranking_sum_by_city = self.data.groupby('city').sum()['ranking']
        self.data.loc[:, 'ranking_summated'] = self.data.loc[:, 'city'].\
            apply(lambda city: ranking_sum_by_city[city])    
    #==============================================================================================
        pf = PolynomialFeatures(3)
        poly_features = pf.fit_transform(
            self.data.loc[:, ['rev_number', 'ranking']]
        )
    
        self.data = pd.concat([
             self.data,
             pd.DataFrame(poly_features)
             ],
             axis=1
        )
    
    
    def clean(self):
    # ################### 5. Clean #################################################### 
        # убираем признаки которые еще не успели обработать, 
        # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
        object_columns = [s for s in self.data.columns if self.data[s].dtypes == 'object']
        self.data.drop(object_columns, axis = 1, inplace=True)
    

#### Запускаем и проверяем, что получилось

In [115]:
df_preproc = PreprocData(data)

df_preproc.preprocess()

df_preproc.fill_missing()

df_preproc.encode()

df_preproc.add_features()

df_preproc.clean()

df_preproc = df_preproc.data
df_preproc.sample(10)

In [116]:
df_preproc.info()

### Посмотрим визуализацию корреляций в получившемся датафрейме:

In [117]:
plt.figure(figsize = (25, 25))

sns.heatmap(df_preproc.corr(), annot = True, cmap = 'Spectral_r');

### Удалим сильно кореллирующие признаки и еще раз визуализируем корелляции в данных:

In [118]:
df_model = df_preproc.drop([
    'population', 'n_of_rest', '_London', 'pr_nan', 'sample',
     0, 1, 2, 3, 5, 8, 9 
],
     axis=1
)
# 3, 5, 6, 8, 9
plt.figure(figsize = (15, 15))
sns.heatmap(df_model.corr(), annot = True, cmap = 'Spectral_r');
df_model['sample'] = df_preproc['sample']

In [119]:
df_model.sample(5)

In [120]:
# Теперь выделим тестовую часть
train_data = df_model.query('sample == 1').drop(['sample'], axis=1)
test_data = df_model.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values            # наш таргет
X = train_data.drop(['rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [121]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [122]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [123]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [124]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [125]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [126]:
y_pred = (y_pred*2).round()/2

In [127]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [128]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh', log=True);

# Submission
Если все устраевает - готовим Submission на kaggle

In [129]:
test_data.sample(10)

In [130]:
test_data = test_data.drop(['rating'], axis=1)

In [131]:
sample_submission

In [132]:
predict_submission = model.predict(test_data)
predict_submission = (predict_submission*2).round()/2

In [133]:
predict_submission

In [134]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
